In [1]:
import os
import sys
project_root = os.getcwd()
sys.path.append(project_root + '/modules')
# Pipeline의 구조를 이해하기 쉽게 하기 위해 작성한 ipynb 파일입니다
from modules.Pipeline import pipe_eval
pipe = pipe_eval(verbose = True) # verbose를 False로 하면 print가 되지 않습니다.
# collection_name = 'chrdb.db',
# persist_directory = 'DB',
# mode = 'NaverCloudEmb'
# topk = 5
# verbose = True : answer과 score을 보여줍니다. 보고싶지 않다면 False로 하면 됩니다.
# default로 지정돼 있습니다. 클래스 선언 때 건네주면 바뀝니다.
pipe.setup(model = 'GPT')
#                   가능한 모델 (default GPT)
#                   1. GPT
#                   2. Exaone
#                   3. Qwen14B
#                   4. Qwen32B
#                   5. HyperClova

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
생성된 DB가 없어 만듭니다.
Received 388 documents for embedding.


/data/ephemeral/home/level4-nlp-finalproject-hackathon-nlp-09-lv3/src/modules/DB/chromadb_storing.py:81: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  self.db = Chroma(


Adding batch 1 of 4...
Adding batch 2 of 4...
Adding batch 3 of 4...
Adding batch 4 of 4...


/data/ephemeral/home/level4-nlp-finalproject-hackathon-nlp-09-lv3/src/modules/DB/chromadb_storing.py:98: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  self.db.persist()


Collection 'chrdb.db' created and persisted at 'modules/DB/Eval_DB_NaverCloudEmb'.
Total documents in collection 'chrdb.db': 388


In [3]:
# query = 'SK하이닉스의 2025~2027년 주주환원 정책에서 변경된 고정배당금과 추가 배당금의 활용 방안은 무엇인가요?'
query = 'NAVER의 분기별 실적 추이 및 전망에 대하여 알려줘'
# validation데이터셋의 첫번째 question
retrieval_result = pipe.Q(query, mode = 'ensemble')
# bm25, dpr, ensemble 가능 (default 'ensemble')


page_content='교보증권의 분기별 실적 추이를 나타내는 표이다. 각 분기별로 매출액, 영업이익, 순이익 등 주요 지표들이 표시되어 있으며, 이를 통해 해당 기업의 경영 성과를 파악할 수 있다.' metadata={'id': 30, 'type': 'paragraph', 'image_route': nan, 'dir_route': 'datas/네이버_교보증권(2024.11.11)', 'file_name': '네이버_교보증권(2024.11.11).pdf', 'page': 2, 'investment': '교보증권(2024.11.11)', 'company_name': '네이버', 'table': nan, 'original_content': '1Q22 2Q22 3Q22 4Q22 1Q23 2Q23 3Q23 4Q23 1Q24 2Q24 3Q24'}
page_content='카카오뱅크가 자산 성장 지속과 함께 2030년 ROE 15% 목표를 달성하려면 당기순이익이 1.3조 원까지 증가해야 하며, 이는 24년 예상이익 대비 198.6% 증가와 연평균 20% 수준의 이익 성장률을 의미한다. 이는 동사가 언급한 영업이익의 연평균 15% 이상 성장 목표를 상회하는 수준이다.' metadata={'id': 57, 'type': 'paragraph', 'image_route': nan, 'dir_route': 'datas/카카오뱅크_삼성증권(2024.11.27)', 'file_name': '카카오뱅크_삼성증권(2024.11.27).pdf', 'page': 4, 'investment': '삼성증권(2024.11.27)', 'company_name': '카카오뱅크', 'table': nan, 'original_content': '동사가 자산 성장의 지속과 더불어 2030년 ROE 15% 목표를 달성하기 위해서는 동사 당기순이익이 1.3 조원까지 증가할 필요가 있음

In [3]:
print(retrieval_result[1].metadata['table'])

nan


In [4]:
answer = pipe.A(query, retrieval_result)
# query 와 retrieval 결과를 받아 answer을 냅니다.



다음 참고 문서를 반드시 활용하여 질문에 대한 정확하고 간결한 답변을 작성하세요. 
답변은 반드시 질문과 관련된 사실에 기반하며, 불필요한 반복이나 모호함 없이 핵심 내용을 포함해야 합니다. 
200자를 초과하지 않도록 작성하세요.

질문: SK하이닉스의 2025~2027년 주주환원 정책에서 변경된 고정배당금과 추가 배당금의 활용 방안은 무엇인가요?
참고 문서1: SK하이닉스는 장 종료 후에 향후 3개년간 주주환원 쟁책을 공시했다. 기존 3개년 주주환 원 정책에서 변화가 생겼다. 변경된 부분의 핵심은 연간 고정배당금을 기존 1,200원에서 1,500원으로 상향하고, 기존 정책 하에서 지급하던 추가 배당금이었던 연간 Free Cash Flow의 5% 재원을 재무 건전성 강화에 우선 활용한다는 점이다. 아울러 정책 종료 후 총 재원 안에서 재무 건전성 목표가 달성되면 추가 환원을 실행하거나, 유의미한 수준의 Free Cash Flow 창출이 예상되는 경우에 조기 환원도 가능하다고 언급했다. 다시 설명하 자면 2025~2027년까지 3년 누적 Free Cash Flow의 50% 수준을 총 재원으로 설정하는 것은 기존과 동일하다. 다만, 기존에 매년 지급하던 FCF 내 5%의 추가 배당금을 앞으로 는 재무 건전성 강화로 사용하는 대신에 정책 종료 후 FCF 50% 수준 안에서 여유 재원을 기반으로 한 추가 환원을 결정한 것이다. 메모리 업황 특성상 변동성이 크기 때문에 예측 가능한 주주환원 규모를 선정하기 어렵기 때문에 3년 누적의 Free Cash Flow 선정은 동종 업체들의 정책과도 일맥상통하는 부분이다. 또한, SK하이닉스의 재무 건전성이 경쟁사들보 다 열위에 있던 점을 고려하면 금번 주주환원 정책 발표는 합리적인 결정이라 판단한다.
참고 문서2: 기업가치 제고계획 목표 4. ’24~’25년 주주환원율 50% 도달 이후 ’27년부터 DPS 유지 또 는 점진적 상향 – 배당 중심 주주환원 정책 예상: 동사의 주주환원 정책은 자사주 매입 및 소각보다는 배당 확대

In [7]:
answer

'2025~2027년 SK하이닉스 주주환원 정책은 연간 고정배당금 1,500원을 지급하고, 3년 누적 Free Cash Flow(Fcf)의 50%를 활용해 추가 환원을 진행할 계획입니다.'

In [5]:
answer = pipe.QA(query, mode = 'ensemble')
# QA를 따로 내는 것이 아니라 한번에 정답을 얻을 수도 있습니다. 이후 정량평가에서는 QA를 사용하면 됩니다.

page_content='SK하이닉스는 향후 3개년간 주주환원 정책을 공시하며, 연간 고정배당금을 1,500원으로 상향하고,  Free Cash Flow의 5% 재원을 재무 건전성 강화에 우선 활용한다고 밝혔다. 이는 메모리 업황 특성상 변동성이 큰 상황에서 내린 합리적인 결정이라고 판단된다.' metadata={'company_name': 'SK하이닉스', 'dir_route': 'datas/SK하이닉스_하나증권(2024.11.28)', 'file_name': 'SK하이닉스_하나증권(2024.11.28).pdf', 'id': 17, 'investment': '하나증권(2024.11.28)', 'original_content': 'SK하이닉스는 장 종료 후에 향후 3개년간 주주환원 쟁책을 공시했다. 기존 3개년 주주환 원 정책에서 변화가 생겼다. 변경된 부분의 핵심은 연간 고정배당금을 기존 1,200원에서 1,500원으로 상향하고, 기존 정책 하에서 지급하던 추가 배당금이었던 연간 Free Cash Flow의 5% 재원을 재무 건전성 강화에 우선 활용한다는 점이다. 아울러 정책 종료 후 총 재원 안에서 재무 건전성 목표가 달성되면 추가 환원을 실행하거나, 유의미한 수준의 Free Cash Flow 창출이 예상되는 경우에 조기 환원도 가능하다고 언급했다. 다시 설명하 자면 2025~2027년까지 3년 누적 Free Cash Flow의 50% 수준을 총 재원으로 설정하는 것은 기존과 동일하다. 다만, 기존에 매년 지급하던 FCF 내 5%의 추가 배당금을 앞으로 는 재무 건전성 강화로 사용하는 대신에 정책 종료 후 FCF 50% 수준 안에서 여유 재원을 기반으로 한 추가 환원을 결정한 것이다. 메모리 업황 특성상 변동성이 크기 때문에 예측 가능한 주주환원 규모를 선정하기 어렵기 때문에 3년 누적의 Free Cash Flow 선정은 동종 업체들의 정책과도 일맥상통하는 부분이다. 또한, SK하이닉스의 재무 건전

In [4]:
generated_answer, G_retrieval_score, G_generation_score = pipe.QA_eval(mode = 'ensemble', sampling = True)

# validation 데이터셋의 무작위 샘플을 가져와 G-eval로 평가할 수 있습니다.

page_content='한화솔루션의 신재생에너지 부문은 4분기 매출액 1조 원, 영업이익 520억 원으로 흑자 전환 예상됨. 다만, 미국 내 과잉 재고와 시장 가격 약세로 조기 수익성 확보는 어려울 것으로 전망함. 동남아시아 및 대중국 태양광 관세 부과와 금리 인하 이후 미국 내 태양광 발전 수요 증가를 회복 변수로 판단함.' metadata={'company_name': '한화솔루션', 'dir_route': 'datas/한화솔루션_메리츠증권(2024.10.31)', 'file_name': '한화솔루션_메리츠증권(2024.10.31).pdf', 'id': 10, 'investment': '메리츠증권(2024.10.31)', 'original_content': '당사의 기존 신재생에너지부문 4Q24 추정은 매출액 1.0조원, 영업이익 520억 원으로 흑자전환은 가능할 점. AMPC 1,613억원을 추정, 이를 제외한 영업적자 는 -1,093억원. 미국 달튼 공장의 생산량 소폭 증가, 반면 미국 내 과잉재고, 시장가격 약세흐름에 조기 수익성 확보는 다소 요원해질 점. 태양광 업황 변수 는 2025년 동남아/대중 태양광 관세부과에 시장가격 상승 가능성, 금리인하 이 후 미국 내 태양광 발전수요 증가에 생산현지화의 이점 반영 가능성 등을 회복 변수로 판단', 'page': 1, 'type': 'paragraph'}
page_content='한화솔루션은 분기 영업실적에서 매출액 2.7조원, 영업적자 -810억원을 기록하며 이는 신재생부문 EPC 자산매각 규모가 가이던스에 미치지 못한 것이 원인이다.' metadata={'company_name': '한화솔루션', 'dir_route': 'datas/한화솔루션_메리츠증권(2024.10.31)', 'file_name': '한화솔루션_메리츠증권(2024.10.31).pdf',

In [5]:
eval_df = pipe.QA_eval(mode = 'ensemble', sampling = False) # sampling이 False일 경우, 모든 validation 데이터셋에 대해 평가를 진행합니다.
# 구현이 귀찮아서 일단 ./datas/g_eval_result_{model_name}.csv 경로에 저장되도록 하였습니다..

267it [1:42:57, 23.14s/it]

267 에 대한 평가를 마쳤습니다.


In [6]:
import pandas as pd
# retrieval G eval score과 generation G eval score의 점수 부분만 파싱합니다. (저장된 csv에는 전부 문자로 돼 있습니다.)

eval_df['generation_score'] = eval_df['generation_score'].apply(lambda x: x[x.rfind('/')-3:x.rfind('/')])
eval_df['retrieval_score'] = eval_df['retrieval_score'].apply(lambda x: x[x.rfind('/')-3:x.rfind('/')])
eval_df['generation_score'] = pd.to_numeric(eval_df['generation_score'], errors = 'coerce')
eval_df['retrieval_score'] = pd.to_numeric(eval_df['retrieval_score'], errors = 'coerce')

In [7]:
eval_df['generation_score'].describe()

count    262.000000
mean      24.194656
std        3.470220
min       12.000000
25%       22.000000
50%       26.000000
75%       26.000000
max       30.000000
Name: generation_score, dtype: float64

In [6]:
retrieval_result[0].metadata['type']

'paragraph'

In [8]:
table = """|---------------------|--------|--------|-------|-------|-------|
| 매출액              | 1,829  | 1,749  | 1,906 | 1,706 | 1,936 |
| 영업이익            | 230    | 83     | 106   | 123   | 139   |
| (증감률)            | -58.5  | -63.9  | 27.8  | 15.9  | 13.0  |
| 지배주주순이익      | -      | -      | -     | -     | -     |
| EPS                 | 10.73  | 10.71  | 11.5  | 12.0  | 12.5  |
| PER (H/L)           | 14.5   | 38.9   | 24.3  | 11.5  | 10.5  |
| PBR (H/L)           | 1.40   | 0.78   | 0.73  | -     | -     |
| EV/EBITDA (H/L)     | -      | -      | -     | -     | -     |
| 영업이익률          | -      | -      | -     | -     | -     |
| ROE                 | -      | -      | -     | -     | -     |"""

In [11]:
import pandas as pd
df = pd.read_csv('/data/ephemeral/home/clone/src/modules/datas/SK케미칼_DB금융투자(2024.11.06)/SK케미칼_DB금융투자(2024.11.06).csv')

In [21]:
import pandas as pd
import re

def markdown_to_df(markdown_table: str) -> pd.DataFrame:
    """
    마크다운 테이블을 Pandas DataFrame으로 변환하는 함수
    
    :param markdown_table: 마크다운 형식의 테이블 문자열
    :return: 변환된 Pandas DataFrame
    """
    lines = markdown_table.strip().split("\n")
    
    # 헤더 추출
    headers = [h.strip() for h in lines[0].split("|") if h.strip()]
    
    # 데이터 행 추출 (두 번째 줄은 구분선이므로 제외)
    data_rows = lines[2:]
    
    # 데이터 처리
    data = []
    for row in data_rows:
        values = [v.strip() for v in row.split("|") if v.strip()]
        data.append(values)
    
    # 데이터프레임 생성
    df = pd.DataFrame(data, columns=headers)
    return df

In [28]:
df = pd.read_csv('/data/ephemeral/home/clone/src/modules/datas/SK케미칼_DB금융투자(2024.11.06)/SK케미칼_DB금융투자(2024.11.06).csv')



In [39]:
q = 'SK하이닉스의 2024년 핵심사업에 대해 알려줘.'
result = pipe.Q(query)

page_content='SK하이닉스는 향후 3개년간 주주환원 정책을 공시하며, 연간 고정배당금을 1,500원으로 상향하고,  Free Cash Flow의 5% 재원을 재무 건전성 강화에 우선 활용한다고 밝혔다. 이는 메모리 업황 특성상 변동성이 큰 상황에서 내린 합리적인 결정이라고 판단된다.' metadata={'company_name': 'SK하이닉스', 'dir_route': 'datas/SK하이닉스_하나증권(2024.11.28)', 'file_name': 'SK하이닉스_하나증권(2024.11.28).pdf', 'id': 17, 'investment': '하나증권(2024.11.28)', 'original_content': 'SK하이닉스는 장 종료 후에 향후 3개년간 주주환원 쟁책을 공시했다. 기존 3개년 주주환 원 정책에서 변화가 생겼다. 변경된 부분의 핵심은 연간 고정배당금을 기존 1,200원에서 1,500원으로 상향하고, 기존 정책 하에서 지급하던 추가 배당금이었던 연간 Free Cash Flow의 5% 재원을 재무 건전성 강화에 우선 활용한다는 점이다. 아울러 정책 종료 후 총 재원 안에서 재무 건전성 목표가 달성되면 추가 환원을 실행하거나, 유의미한 수준의 Free Cash Flow 창출이 예상되는 경우에 조기 환원도 가능하다고 언급했다. 다시 설명하 자면 2025~2027년까지 3년 누적 Free Cash Flow의 50% 수준을 총 재원으로 설정하는 것은 기존과 동일하다. 다만, 기존에 매년 지급하던 FCF 내 5%의 추가 배당금을 앞으로 는 재무 건전성 강화로 사용하는 대신에 정책 종료 후 FCF 50% 수준 안에서 여유 재원을 기반으로 한 추가 환원을 결정한 것이다. 메모리 업황 특성상 변동성이 크기 때문에 예측 가능한 주주환원 규모를 선정하기 어렵기 때문에 3년 누적의 Free Cash Flow 선정은 동종 업체들의 정책과도 일맥상통하는 부분이다. 또한, SK하이닉스의 재무 건전

In [44]:
for i in result:
    print(i.metadata['type'])

paragraph
paragraph
paragraph
paragraph
paragraph


In [9]:
print(answer1)

다음 참고 문서를 반드시 활용하여 질문에 대한 정확하고 간결한 답변을 작성하세요. 
답변은 반드시 질문과 관련된 사실에 기반하며, 불필요한 반복이나 모호함 없이 핵심 내용을 포함해야 합니다. 
200자를 초과하지 않도록 작성하세요.

질문: SK하이닉스의 2025~2027년 주주환원 정책에서 변경된 고정배당금과 추가 배당금의 활용 방안은 무엇인가요?
참고 문서1: SK하이닉스는 장 종료 후에 향후 3개년간 주주환원 쟁책을 공시했다. 기존 3개년 주주환 원 정책에서 변화가 생겼다. 변경된 부분의 핵심은 연간 고정배당금을 기존 1,200원에서 1,500원으로 상향하고, 기존 정책 하에서 지급하던 추가 배당금이었던 연간 Free Cash Flow의 5% 재원을 재무 건전성 강화에 우선 활용한다는 점이다. 아울러 정책 종료 후 총 재원 안에서 재무 건전성 목표가 달성되면 추가 환원을 실행하거나, 유의미한 수준의 Free Cash Flow 창출이 예상되는 경우에 조기 환원도 가능하다고 언급했다. 다시 설명하 자면 2025~2027년까지 3년 누적 Free Cash Flow의 50% 수준을 총 재원으로 설정하는 것은 기존과 동일하다. 다만, 기존에 매년 지급하던 FCF 내 5%의 추가 배당금을 앞으로 는 재무 건전성 강화로 사용하는 대신에 정책 종료 후 FCF 50% 수준 안에서 여유 재원을 기반으로 한 추가 환원을 결정한 것이다. 메모리 업황 특성상 변동성이 크기 때문에 예측 가능한 주주환원 규모를 선정하기 어렵기 때문에 3년 누적의 Free Cash Flow 선정은 동종 업체들의 정책과도 일맥상통하는 부분이다. 또한, SK하이닉스의 재무 건전성이 경쟁사들보 다 열위에 있던 점을 고려하면 금번 주주환원 정책 발표는 합리적인 결정이라 판단한다.
참고 문서2: 이번에 발표된 향후 3년간 주주환원 정책과 기업가치 제고 계획에 대해 다소 실망스러운 시장 반응이 있을 수 있다. 다만, 메모리 업종은 주주환원보다 기술 리더십이 우선시 되어 야 한다. 그럼에도 불구하고 변동성 높은

In [2]:
import os
import sys
project_root = os.getcwd()
sys.path.append(project_root + '/modules')
from Pipeline import pipe_service

pipe = pipe_service()
pipe.Q('SK하이닉스의 핵심사업에 대해 알려줘.')



생성된 DB가 있어 로드합니다.


/data/ephemeral/home/clone/src/modules/DB/chromadb_storing.py:127: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  self.db = Chroma(


Collection 'chrdb.db' loaded from 'modules/DB/Service_DB_NaverCloudEmb'.
Total documents in collection 'chrdb.db': 973


[Document(metadata={'company_name': 'SK하이닉스', 'dir_route': 'datas/SK하이닉스_한화투자증권(2024.11.27)', 'file_name': 'SK하이닉스_한화투자증권(2024.11.27).pdf', 'id': 31, 'image_route': 'datas/SK하이닉스_한화투자증권(2024.11.27)/page_3_id_31_image.png', 'investment': '한화투자증권(2024.11.27)', 'page': 3, 'type': 'figure'}, page_content='SK하이닉스의 사업부별 분기 매출 비중을 나타내는 그래프입니다. DRAM, NAND, 기타 부문으로 나뉘어 있으며, DRAM 비중이 지속적으로 높은 수치를 보이고 있습니다.'),
 Document(metadata={'id': 9, 'type': 'paragraph', 'image_route': nan, 'dir_route': 'datas/CJ제일배당_메리츠증권(2024.11.19)', 'file_name': 'CJ제일배당_메리츠증권(2024.11.19).pdf', 'page': 1, 'investment': '메리츠증권(2024.11.19)', 'company_name': 'CJ제일배당', 'table': nan, 'original_content': 'CJ제일제당의 바이오 사업 매각 추진 보도. 언론 보도에 따른 예상 매각 가치는 5~6조원, 실제 매각 추진 과정에서 가치 변화는 가능하지만 매각 추진 이슈 자체 로도 긍정적인 뉴스. 지난해부터 이어진 비핵심 계열사 지상쥐(중국 식품), 셀렉타 (SPC, 농축대두단백)의 매각으로 식품, 그 중에서도 K푸드 중심으로 선택과 집중 하는 의사결정이 시작. 바이오 사업의 매각을 통해 1차적으로 재무구조가 개선되 고, 2차